In [1]:
import tensorflow as tf
import keras
from keras.utils import to_categorical
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from keras import models
from keras import layers
from keras import utils
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import wandb
from wandb.keras import WandbCallback

Using TensorFlow backend.


In [7]:
wandb.init()
config = wandb.config

config.max_len = 11
config.buckets = 20
#max_len=11
#buckets=20

labels=['background noise', 'backward', 'bed', 'bird', 'cat', 'dog', 'eight', 'five', 'follow', 'forward', 'four', 'go', 'happy', 'house', 'learn',
        'left', 'marvin', 'nine', 'no', 'off', 'on', 'one', 'right', 'seven', 'sheila', 'six', 'stop', 'three', 'tree', 'two', 'up', 'visual', 'wow', 'yes', 'zero']
indices=[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,34]

PATH = "./npy_data/"
def get_train_test(split_ratio=0.8, random_state=42):
    # Get available labels
    #labels, indices, _ = get_labels(DATA_PATH)

    # Getting first arrays
    
    X = np.load(PATH+labels[0] + '.npy')
    y = np.zeros(X.shape[0])

    # Append all of the dataset into one single array, same goes for y
    for i, label in enumerate(labels[1:]):
        x = np.load(PATH+label + '.npy')
        X = np.vstack((X, x))
        y = np.append(y, np.full(x.shape[0], fill_value= (i + 1)))

    assert X.shape[0] == len(y)

    return train_test_split(X, y, test_size= (1 - split_ratio), random_state=random_state, shuffle=True)


# # Loading train set and test set
X_train, X_test, y_train, y_test = get_train_test()

# # Feature dimension
channels = 1
config.epochs = 500
#config.batch_size = 100
num_classes = 35

X_train = X_train.reshape(X_train.shape[0], config.buckets, config.max_len, channels)
X_test = X_test.reshape(X_test.shape[0], config.buckets, config.max_len, channels)

y_train_hot = to_categorical(y_train)
y_test_hot = to_categorical(y_test)

# build model
model = keras.Sequential()
model.add(layers.Conv2D(32,(3,3),input_shape=(config.buckets, config.max_len,channels),
    activation='relu'))
model.add(layers.Conv2D(32,(3,3),activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Flatten())
model.add(layers.Dropout(0.2))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(num_classes, activation='softmax'))

model.compile(loss="categorical_crossentropy",
                  optimizer="adam",
                  metrics=['accuracy'])

wandb.init()
model.fit(X_train, y_train_hot, epochs=config.epochs
          , validation_data=(X_test, y_test_hot),callbacks=[WandbCallback(data_type="image", labels=labels)])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.




Train on 76094 samples, validate on 19024 samples
Epoch 1/500
76094/76094 [==============================] - 20s 263us/step - loss: 2.1186 - accuracy: 0.4045 - val_loss: 1.2305 - val_accuracy: 0.6395
Epoch 2/500
76094/76094 [==============================] - 25s 323us/step - loss: 1.3553 - accuracy: 0.6003 - val_loss: 1.0288 - val_accuracy: 0.7019
Epoch 3/500
76094/76094 [==============================] - 19s 244us/step - loss: 1.1839 - accuracy: 0.6478 - val_loss: 0.9415 - val_accuracy: 0.7223
Epoch 4/500
76094/76094 [==============================] - 22s 285us/step - loss: 1.0992 - accuracy: 0.6733 - val_loss: 0.8975 - val_accuracy: 0.7341
Epoch 5/500
76094/76094 [==============================] - 26s 338us/step - loss: 1.0433 - accuracy: 0.6858 - val_loss: 0.8692 - val_accuracy: 0.7431
Epoch 6/500
76094/76094 [==============================] - 23s 309us/step - loss: 1.0068 - accuracy: 0.6960 - val_loss: 0.8427 - val_accuracy: 0.7524
Epoch 7/500
76094/76094 [========================

76094/76094 [==============================] - 20s 260us/step - loss: 0.7423 - accuracy: 0.7727 - val_loss: 0.8106 - val_accuracy: 0.7660
Epoch 108/500
76094/76094 [==============================] - 20s 263us/step - loss: 0.7403 - accuracy: 0.7730 - val_loss: 0.8519 - val_accuracy: 0.7570
Epoch 109/500
76094/76094 [==============================] - 20s 261us/step - loss: 0.7390 - accuracy: 0.7721 - val_loss: 0.8231 - val_accuracy: 0.7639
Epoch 110/500
76094/76094 [==============================] - 20s 261us/step - loss: 0.7396 - accuracy: 0.7719 - val_loss: 0.8069 - val_accuracy: 0.7643
Epoch 111/500
76094/76094 [==============================] - 20s 264us/step - loss: 0.7346 - accuracy: 0.7725 - val_loss: 0.8045 - val_accuracy: 0.7669
Epoch 112/500
76094/76094 [==============================] - 20s 262us/step - loss: 0.7348 - accuracy: 0.7737 - val_loss: 0.8005 - val_accuracy: 0.7665
Epoch 113/500
76094/76094 [==============================] - 20s 261us/step - loss: 0.7387 - accuracy:

Epoch 161/500
76094/76094 [==============================] - 24s 319us/step - loss: 0.7245 - accuracy: 0.7784 - val_loss: 0.7954 - val_accuracy: 0.7728
Epoch 162/500
76094/76094 [==============================] - 24s 318us/step - loss: 0.7195 - accuracy: 0.7794 - val_loss: 0.8217 - val_accuracy: 0.7611
Epoch 163/500
76094/76094 [==============================] - 24s 318us/step - loss: 0.7253 - accuracy: 0.7779 - val_loss: 0.8037 - val_accuracy: 0.7668
Epoch 164/500
76094/76094 [==============================] - 24s 319us/step - loss: 0.7312 - accuracy: 0.7763 - val_loss: 0.8169 - val_accuracy: 0.7633
Epoch 165/500
76094/76094 [==============================] - 25s 328us/step - loss: 0.7226 - accuracy: 0.7767 - val_loss: 0.8206 - val_accuracy: 0.7646
Epoch 166/500
76094/76094 [==============================] - 25s 323us/step - loss: 0.7268 - accuracy: 0.7743 - val_loss: 0.8039 - val_accuracy: 0.7680
Epoch 167/500
76094/76094 [==============================] - 25s 323us/step - loss: 0.71

Epoch 215/500
76094/76094 [==============================] - 25s 328us/step - loss: 0.7162 - accuracy: 0.7806 - val_loss: 0.8186 - val_accuracy: 0.7678
Epoch 216/500
76094/76094 [==============================] - 24s 322us/step - loss: 0.7166 - accuracy: 0.7802 - val_loss: 0.7943 - val_accuracy: 0.7717
Epoch 217/500
76094/76094 [==============================] - 26s 341us/step - loss: 0.7150 - accuracy: 0.7812 - val_loss: 0.8063 - val_accuracy: 0.7710
Epoch 218/500
76094/76094 [==============================] - 25s 328us/step - loss: 0.7136 - accuracy: 0.7819 - val_loss: 0.8151 - val_accuracy: 0.7690
Epoch 219/500
76094/76094 [==============================] - 25s 325us/step - loss: 0.7160 - accuracy: 0.7811 - val_loss: 0.8244 - val_accuracy: 0.7670
Epoch 220/500
76094/76094 [==============================] - 25s 325us/step - loss: 0.7149 - accuracy: 0.7817 - val_loss: 0.8200 - val_accuracy: 0.7665
Epoch 221/500
76094/76094 [==============================] - 27s 356us/step - loss: 0.71

Epoch 269/500
76094/76094 [==============================] - 22s 295us/step - loss: 0.7141 - accuracy: 0.7803 - val_loss: 0.8217 - val_accuracy: 0.7676
Epoch 270/500
76094/76094 [==============================] - 22s 286us/step - loss: 0.7107 - accuracy: 0.7821 - val_loss: 0.8219 - val_accuracy: 0.7663
Epoch 271/500
76094/76094 [==============================] - 22s 294us/step - loss: 0.7006 - accuracy: 0.7858 - val_loss: 0.8355 - val_accuracy: 0.7639
Epoch 272/500
76094/76094 [==============================] - 23s 298us/step - loss: 0.7113 - accuracy: 0.7819 - val_loss: 0.8305 - val_accuracy: 0.7630
Epoch 273/500
76094/76094 [==============================] - 22s 288us/step - loss: 0.7120 - accuracy: 0.7817 - val_loss: 0.8151 - val_accuracy: 0.7702
Epoch 274/500
76094/76094 [==============================] - 21s 275us/step - loss: 0.7090 - accuracy: 0.7846 - val_loss: 0.7953 - val_accuracy: 0.7707
Epoch 275/500
76094/76094 [==============================] - 24s 313us/step - loss: 0.70

Epoch 323/500
76094/76094 [==============================] - 21s 273us/step - loss: 0.6978 - accuracy: 0.7866 - val_loss: 0.7879 - val_accuracy: 0.7735
Epoch 324/500
76094/76094 [==============================] - 21s 274us/step - loss: 0.7102 - accuracy: 0.7819 - val_loss: 0.8197 - val_accuracy: 0.7673
Epoch 325/500
76094/76094 [==============================] - 22s 291us/step - loss: 0.7058 - accuracy: 0.7845 - val_loss: 0.8032 - val_accuracy: 0.7690
Epoch 326/500
76094/76094 [==============================] - 21s 282us/step - loss: 0.6958 - accuracy: 0.7869 - val_loss: 0.8229 - val_accuracy: 0.7693
Epoch 327/500
76094/76094 [==============================] - 21s 275us/step - loss: 0.7050 - accuracy: 0.7853 - val_loss: 0.8135 - val_accuracy: 0.7672
Epoch 328/500
76094/76094 [==============================] - 21s 278us/step - loss: 0.7063 - accuracy: 0.7853 - val_loss: 0.8327 - val_accuracy: 0.7652
Epoch 329/500
76094/76094 [==============================] - 22s 292us/step - loss: 0.70

Epoch 377/500
76094/76094 [==============================] - 28s 373us/step - loss: 0.6946 - accuracy: 0.7872 - val_loss: 0.8274 - val_accuracy: 0.7691
Epoch 378/500
76094/76094 [==============================] - 26s 339us/step - loss: 0.7014 - accuracy: 0.7866 - val_loss: 0.8374 - val_accuracy: 0.7657
Epoch 379/500
76094/76094 [==============================] - 25s 333us/step - loss: 0.6997 - accuracy: 0.7857 - val_loss: 0.8276 - val_accuracy: 0.7653
Epoch 380/500
76094/76094 [==============================] - 24s 320us/step - loss: 0.6993 - accuracy: 0.7871 - val_loss: 0.8134 - val_accuracy: 0.7738
Epoch 381/500
76094/76094 [==============================] - 24s 322us/step - loss: 0.7037 - accuracy: 0.7861 - val_loss: 0.8042 - val_accuracy: 0.7710
Epoch 382/500
76094/76094 [==============================] - 24s 318us/step - loss: 0.7035 - accuracy: 0.7859 - val_loss: 0.8168 - val_accuracy: 0.7693
Epoch 383/500
76094/76094 [==============================] - 24s 321us/step - loss: 0.70

76094/76094 [==============================] - 40s 519us/step - loss: 0.6892 - accuracy: 0.7908 - val_loss: 0.8096 - val_accuracy: 0.7723
Epoch 484/500
76094/76094 [==============================] - 27s 349us/step - loss: 0.6947 - accuracy: 0.7898 - val_loss: 0.8318 - val_accuracy: 0.7640
Epoch 485/500
76094/76094 [==============================] - 26s 344us/step - loss: 0.6945 - accuracy: 0.7882 - val_loss: 0.8228 - val_accuracy: 0.7672
Epoch 486/500
76094/76094 [==============================] - 26s 345us/step - loss: 0.7083 - accuracy: 0.7872 - val_loss: 0.8257 - val_accuracy: 0.7719
Epoch 487/500
76094/76094 [==============================] - 26s 345us/step - loss: 0.7016 - accuracy: 0.7876 - val_loss: 0.8274 - val_accuracy: 0.7634
Epoch 488/500
76094/76094 [==============================] - 26s 345us/step - loss: 0.7019 - accuracy: 0.7869 - val_loss: 0.8462 - val_accuracy: 0.7588
Epoch 489/500
76094/76094 [==============================] - 27s 349us/step - loss: 0.6986 - accuracy:

In [8]:
# Save JSON config to disk
json_config = model.to_json()
with open('IMP_1000_model.json', 'w') as json_file:
    json_file.write(json_config)
# Save weights to disk
model.save_weights('IMP_1000_model_weights.h5')
